# Classification Algorithms and Resampling Methods Part 2

In [1]:
import numpy as np
import pandas as pd
import pickle
import datetime

import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import RandomOverSampler, SMOTE

from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

import warnings
warnings.filterwarnings(action = 'ignore')
import gc

In [2]:
with open('lending_club_modeling_pick.pkl', 'rb') as pickle_file:
    df = pickle.load(pickle_file)

In [3]:
pd.set_option('display.float_format', lambda x: '%.5f' % x)
#pd.reset_option('display.float_format')
pd.set_option('display.max_columns', None)

In [4]:
df.head()

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,annual_inc,verification_status,issue_d,pymnt_plan,purpose,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,application_type,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,disbursement_method,debt_settlement_flag,TARGET
0,3600.00000,3600.00000,3600.00000,36,13.99000,123.03000,C,C4,10,MORTGAGE,55000.00000,Not Verified,16770.00000,n,debt_consolidation,PA,5.91000,0.00000,12265.00000,675.00000,679.00000,1.00000,30.00000,72.31284,7.00000,0.00000,2765.00000,29.70000,13.00000,w,0.00000,0.00000,4421.72392,4421.72000,3600.00000,821.72000,0.00000,0.00000,0.00000,17897.00000,122.67000,18017.00000,17956.00000,564.00000,560.00000,0.00000,30.00000,Individual,0.00000,722.00000,144904.00000,2.00000,2.00000,0.00000,1.00000,21.00000,4981.00000,36.00000,3.00000,3.00000,722.00000,34.00000,9300.00000,3.00000,1.00000,4.00000,4.00000,20701.00000,1506.00000,37.20000,0.00000,0.00000,148.00000,128.00000,3.00000,3.00000,1.00000,4.00000,69.00000,4.00000,69.00000,2.00000,2.00000,4.00000,2.00000,5.00000,3.00000,4.00000,9.00000,4.00000,7.00000,0.00000,0.00000,0.00000,3.00000,76.90000,0.00000,0.00000,0.00000,178050.00000,7746.00000,2400.00000,13734.00000,N,Cash,N,0
1,24700.00000,24700.00000,24700.00000,36,11.99000,820.28000,C,C1,10,MORTGAGE,65000.00000,Not Verified,16770.00000,n,small_business,SD,16.06000,1.00000,10926.00000,715.00000,719.00000,4.00000,6.00000,72.31284,22.00000,0.00000,21470.00000,19.20000,38.00000,w,0.00000,0.00000,25679.66000,25679.66000,24700.00000,979.66000,0.00000,0.00000,0.00000,16953.00000,926.35000,18017.00000,17956.00000,699.00000,695.00000,0.00000,44.16422,Individual,0.00000,0.00000,204396.00000,1.00000,1.00000,0.00000,1.00000,19.00000,18005.00000,73.00000,2.00000,3.00000,6472.00000,29.00000,111800.00000,0.00000,0.00000,6.00000,4.00000,9733.00000,57830.00000,27.10000,0.00000,0.00000,113.00000,192.00000,2.00000,2.00000,4.00000,2.00000,37.00000,0.00000,6.00000,0.00000,5.00000,5.00000,13.00000,17.00000,6.00000,20.00000,27.00000,5.00000,22.00000,0.00000,0.00000,0.00000,2.00000,97.40000,7.70000,0.00000,0.00000,314017.00000,39475.00000,79300.00000,24667.00000,N,Cash,N,0
2,20000.00000,20000.00000,20000.00000,60,10.78000,432.66000,B,B4,10,MORTGAGE,63000.00000,Not Verified,16770.00000,n,home_improvement,IL,10.78000,0.00000,11170.00000,695.00000,699.00000,0.00000,34.54092,72.31284,6.00000,0.00000,7869.00000,56.20000,18.00000,w,0.00000,0.00000,22705.92429,22705.92000,20000.00000,2705.92000,0.00000,0.00000,0.00000,17318.00000,15813.30000,18017.00000,17956.00000,704.00000,700.00000,0.00000,44.16422,Joint App,0.00000,0.00000,189699.00000,0.00000,1.00000,0.00000,4.00000,19.00000,10827.00000,73.00000,0.00

# Conducting Train Test Split

In [4]:
def dummy_encoding(df):
    
    df = pd.get_dummies(df)
    
    return df

In [5]:
def preprocess_inputs(df):
    df = df.copy()
    
    #Binary Encode
    df = dummy_encoding(df)
    
    #split into x and y arrays
    y = df['TARGET']
    X = df.drop('TARGET', axis = 1)
    
    #Train Test Split
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.8, random_state = 1, stratify = y)
    
    #Scale X
    sc = StandardScaler()
    sc.fit(X_train)
    X_train = sc.transform(X_train)
    X_test = sc.transform(X_test)    
    
    return X_train, X_test, y_train, y_test

In [6]:
X_train, X_test, y_train, y_test = preprocess_inputs(df)

In [6]:
X_train.shape, y_train.shape

((1805776, 222), (1805776,))

In [8]:
del df
gc.collect()

60

# Creating A Baseline For Accuracy (Logistic Regression & Random Forest)
### Oversampling with SMOTE

In [7]:
sm = SMOTE(random_state=12, sampling_strategy = 'minority')
X_train_r, y_train_r = sm.fit_resample(X_train, y_train)

In [10]:
X_train_r.shape, y_train_r.shape

((3611488, 222), (3611488,))

### Logistic Regression

In [11]:
clf = LogisticRegression(C = 0.0001, random_state = 21)

clf.fit(X_train_r, y_train_r) 

LogisticRegression(C=0.0001, random_state=21)

In [22]:
y_predict_test = clf.predict(X_test)
print("\n")
print("[Test] Accuracy score (y_predict_test, y_test):",accuracy_score(y_predict_test, y_test))

y_predict_training = clf.predict(X_train_r)
print("\n")
print("[Training] Accuracy score: (y_train_r, y_predict_training)",accuracy_score(y_train_r, y_predict_training))



[Test] Accuracy score (y_predict_test, y_test): 0.9847644785079024


[Training] Accuracy score: (y_train_r, y_predict_training) 0.9925676064824249


In [23]:
print("[Training Classification Report]")
print(classification_report(y_train_r, y_predict_training))

print("[Test Classification Report]")
print(classification_report(y_test, y_predict_test))

[Training Classification Report]
              precision    recall  f1-score   support

           0       1.00      0.99      0.99   1805744
           1       0.99      1.00      0.99   1805744

    accuracy                           0.99   3611488
   macro avg       0.99      0.99      0.99   3611488
weighted avg       0.99      0.99      0.99   3611488

[Test Classification Report]
              precision    recall  f1-score   support

           0       1.00      0.98      0.99    451437
           1       0.00      0.38      0.00         8

    accuracy                           0.98    451445
   macro avg       0.50      0.68      0.50    451445
weighted avg       1.00      0.98      0.99    451445



Recall score seems to have increased significantly but precision is still off.

### Random Forest

In [ ]:
clf_rf = RandomForestClassifier(n_estimators=40, random_state=21)
clf_rf.fit(X_train_r, y_train_r)

# Hyperparameter Tuning

In [ ]:
import lightgbm as lgb

In [ ]:
train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_test, label=y_test, reference=train_data)

In [ ]:
model = lgb.train(#parameters)